# Age Estimation 

In [1]:
import numpy as np
import os
from helperP import *

ImportError: bad magic number in 'helperP': b'B\r\r\n'

# Loading Data

In [ ]:
base_dir = 'DATASET/'

age_train, features_train = prepare_data('train', base_dir)
age_val, features_val = prepare_data('val', base_dir)
_, features_test = prepare_data('test', base_dir)
show_data(base_dir)

# Implement Closed Form Solution
```
Arguments:
    age          -- numpy array, shape (n, )
    features     -- numpy array, shape (n, 2048)
Returns:
    weights      -- numpy array, (2048, )
    bias         -- numpy array, (1, )
```

In [ ]:
def closed_form_solution(age, features):
    # Preprocess
    H = features
    ones = np.ones(len(H))
    H = np.column_stack((ones,H))  # 按列合并
    Y = age

    # n = features.shape[0]
    
    # Define parameter weights
    
    ##########################################################################
    # TODO: YOUR CODE HERE
    ########################################################################## 
    # calculate the closed form solution
    weights = None

    # EW+FB=G   ->  | E F | |W| = |G|
    # HW+IB=J       | H I | |b|   |J|

    weights = np.linalg.inv(H.T.dot(H)).dot(H.T).dot(Y)
    
    # E = np.sum(features*features,axis=1)
    # F = np.sum(features,axis=1).T
    # G = np.sum(np.multiply(features, Y[:, np.newaxis]),axis=1).T
    # H = np.sum(features,axis=1)
    # I = n
    # J = np.sum(Y)
    

    # print(E)
    # a=np.concatenate([E,F],axis=1)
    # b=np.concatenate([H,I],axis=1)
    # K=np.concatenate(a,b,axis=1)
    # L=np.concatenate([G,J])

    # weights=np.linalg.solve(K,L)

    # separate the weights and bias
    bias    = weights[0]
    weights    = weights[1:]
    
    return weights, bias

# Validate

In [ ]:
w, b = closed_form_solution(age_train, features_train)
loss, pred = evaluate(w, b, age_val, features_val)
print("Your validate loss is:", round(loss, 3))

# Test and Generate results file

In [ ]:
prediction = test(w, b, features_test, 'cfs.txt')
print("Test results has saved to cfs.txt")
print(prediction[:10])


# Implement Gradient descent
Gradient descent is an optimization algorithm used to minimize some function by iteratively moving in the direction of steepest descent as defined by the negative of the gradient. In machine learning, we use gradient descent to update the parameters of our model.

```
Arguments:
    age          -- numpy array, label, (n, )
    feature      -- numpy array, features, (n, 2048)
Return:
    weights      -- numpy array, (2048, )
    bias         -- numpy array, (1, )
```

In [ ]:
def gradient_descent(age, feature):
    assert len(age) == len(feature)

    # Init weights and bias
    weights = np.random.randn(2048, 1)
    bias = np.random.randn(1, 1)
    m,n = feature.shape
    # Learning rate
    lr = 10e-3

    # control the times of processing
    for e in range(epoch):
        ##########################################################################
        # TODO: YOUR CODE HERE
        ##########################################################################
        l_w = np.zeros((2048, 1), dtype=float)
        l_b = 0
        
        for i in range(m):
            tmp = feature[i, :].reshape(-1, 1)
            l_w += (np.dot(feature[i, :], weights) + bias - float(age[i])) * tmp
        l_w /= m
        for i in range(m):
            l_b += np.dot(feature[i, :], weights) + bias - float(age[i])
        l_b /= m

        weights -= lr * l_w
        bias -= lr * l_b
        if momentum:
            pass  # You  can also consider the gradient descent with momentum
    return weights, bias

# Train and validate

In [ ]:
w, b = gradient_descent(age_train, features_train)
loss, pred = evaluate(w, b, age_val, features_val)
print("Your validate score is:", round(loss, 3))

In [ ]:
#  Test and Generate results file

In [ ]:
prediction = test(w, b, features_test, 'gd.txt')
print("Test results has saved to gd.txt")
print(prediction[:10])

# Implement Stochastic Gradient descent
Stochastic Gradient Descent (SGD) is a simple yet very efficient approach to discriminative learning of linear classifiers under convex loss functions such as (linear) Support Vector Machines and Logistic Regression. Even though SGD has been around in the machine learning community for a long time, it has received a considerable amount of attention just recently in the context of large-scale learning.
```
Arguments:
    age          -- numpy array, label, (n, )
    feature      -- numpy array, features, (n, 2048)
Return:
    weights      -- numpy array, (2048, )
    bias         -- numpy array, (1, )
```

In [ ]:
def stochastic_gradient_descent(age, feature):
    # check the inputs
    assert len(age) == len(feature)
    
    # Set the random seed
    np.random.seed(0)

    # Init weights and bias
    weights = np.random.rand(2048, 1)
    bias = np.random.rand(1, 1)

    # Learning rate
    lr = 10e-5

    # Batch size
    batch_size = 16
 
    # Number of mini-batches
    t = len(age) // batch_size

    for e in range(epoch_sgd):
        # Shuffle training data
        n = np.random.permutation(len(feature))  
        
        for m in range(t):
            # Providing mini batch with fixed batch size of 16
            batch_feature = feature[n[m * batch_size : (m+1) * batch_size]]
            batch_age = age[n[m * batch_size : (m+1) * batch_size]]
            
            ##########################################################################
            # TODO: YOUR CODE HERE
            ########################################################################## 
            
            l_w = np.zeros((2048, 1), dtype=float)
            l_b = 0
            
            for i in range(batch_size):
                tmp = batch_feature[i, :].reshape(2048, 1)
                l_w += (np.dot(batch_feature[i, :], weights) + bias - float(batch_age[i])) * tmp
            l_w /= batch_size
            
            for i in range(batch_size):
                l_b += np.dot(batch_feature[i, :], weights) + bias - float(batch_age[i])
            l_b /= batch_size
            
            weights -= lr * l_w
            bias -= lr * l_b
   
                
            if momentum:
                pass # You can also consider the gradient descent with momentum
        
        print('=> epoch:', e + 1, '  Loss:', round(loss,4))
    return weights, bias

# Train and validate

In [ ]:
w, b = stochastic_gradient_descent(age_train, features_train)
loss, pred = evaluate(w, b, age_val, features_val)
print("Your validate score is:", round(loss, 3))

# Test and Generate results file

In [ ]:
prediction = test(w, b, features_test, 'sgd.txt')
print("Test results has saved to sgd.txt")
print(prediction[:10])